## ChatGLM For RAG

In [ ]:
!git clone https://github.com/THUDM/ChatGLM3.git

In [ ]:
!pip innstall modelscope

from modelscope import snapshot_download
model_dir = snapshot_download("ZhipuAI/chatglm3-6b", revision = "v1.0.0")

In [ ]:
!pip install -r ./ChatGLM3/requirements.txt 

In [ ]:
!python ./ChatGLM3/openai_api_demo/api_server.py

In [ ]:
from langchain.chains import LLMChain
from langchain.schema.messages import AIMessage
from langchain_community.llms.chatglm3 import ChatGLM3
from langchain_community.embeddings import ModelScopeEmbeddings
from langchain_core.prompts import PromptTemplate

In [ ]:
endpoint_url = "http://0.0.0.0:8000/v1/chat/completions"
llm = ChatGLM3(
    endpoint_url=endpoint_url,
    max_tokens=80000,
    prefix_messages=messages,
    top_p=0.9,
)

model_id = "Jerry0/m3e-base"
embeddings = ModelScopeEmbeddings(model_id=model_id)

template = """{question}"""
prompt = PromptTemplate.from_template(template)